### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

##### Read all the pdf's inside the directory

In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data/pdfs")

Found 5 PDF files to process

Processing: CN Unit 1 Tesseract.pdf
  ✓ Loaded 54 pages

Processing: CN Unit 2 Tesseract.pdf
  ✓ Loaded 34 pages

Processing: CN Unit 3 Tesseract.pdf
  ✓ Loaded 55 pages

Processing: CN Unit 4 Tesseract.pdf
  ✓ Loaded 49 pages

Processing: CN Unit 5 Tesseract.pdf
  ✓ Loaded 33 pages

Total documents loaded: 225


In [5]:
all_pdf_documents

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '..\\data\\pdfs\\CN Unit 1 Tesseract.pdf', 'total_pages': 54, 'page': 0, 'page_label': '1', 'source_file': 'CN Unit 1 Tesseract.pdf', 'file_type': 'pdf'}, page_content='Data Communications: \n Data communications are the exchange of data between two devices via some \nform of transmission medium such as a wire cable. \nThe effectiveness of a data communications system depends on four fundamental \ncharacteristics: \n1. Delivery The system must deliver data to the correct destination. Data \nmust be received by the intended device or user and only by that device or \nuser. \n2. Accuracy The system must deliver the data accurately. Data should not be \naltered. If the data is altered in transmission and left uncorrected are unusable. \n3. Timeliness The system must deliver data in a timely manner. Data delivered late are \nuseless. In the case of video and audio, timely delivery means delivering da

##### Text splitting get into chunks

In [7]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [8]:
chunks=split_documents(all_pdf_documents)
chunks

Split 225 documents into 426 chunks

Example chunk:
Content: Data Communications: 
 Data communications are the exchange of data between two devices via some 
form of transmission medium such as a wire cable. 
The effectiveness of a data communications system d...
Metadata: {'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '..\\data\\pdfs\\CN Unit 1 Tesseract.pdf', 'total_pages': 54, 'page': 0, 'page_label': '1', 'source_file': 'CN Unit 1 Tesseract.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '..\\data\\pdfs\\CN Unit 1 Tesseract.pdf', 'total_pages': 54, 'page': 0, 'page_label': '1', 'source_file': 'CN Unit 1 Tesseract.pdf', 'file_type': 'pdf'}, page_content='Data Communications: \n Data communications are the exchange of data between two devices via some \nform of transmission medium such as a wire cable. \nThe effectiveness of a data communications system depends on four fundamental \ncharacteristics: \n1. Delivery The system must deliver data to the correct destination. Data \nmust be received by the intended device or user and only by that device or \nuser. \n2. Accuracy The system must deliver the data accurately. Data should not be \naltered. If the data is altered in transmission and left uncorrected are unusable. \n3. Timeliness The system must deliver data in a timely manner. Data delivered late are \nuseless. In the case of video and audio, timely delivery means delivering da

##### Embedding and VectorDb

In [11]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


d:\Gen AI\RAG\Simple-RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to reg

Model loaded successfully. Embedding dimension: 384


##### Vector Store